# Training feature selection

## Description

...


## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

import json
from collections import Counter
from pprint import pprint

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D

import numpy as np
import pandas as pd
from joblib import dump

import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import itertools

import warnings  # Import the warnings module

import matplotlib.colors as mcolors

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# Maize, SoyaBeans, Sunflower
# Pasture, Fallow, Tree,
# Wheat, Sorghum, Groundnuts, Lucern,
# WheatMaize,WheatSunflower,WheatSoya
crop_selected = 'WheatSoya'

## Load training data and label dictionary

We will load the training data saved from the [feature extracion notebook](3_Training_feature_extraction.ipynb), along with the mapping between crop labels and the numerical classes.

In [ ]:
dataset_name1 = 'sb16_17';
dataset_name2 = 'sb17_18';
dataset_name3 = 'sb18_19';
dataset_name4 = 'sb19_20';
dataset_name5 = 'sb20_21';

dataset_suffix = '_tree_cleaned';

predictor_column = 6; # Column where we start selecting the predictors
ndvi_threshold = 0; # NDVI min Threshold

: 

In [ ]:

base_path = 'Input'
output_file1 = os.path.join(base_path, f'{dataset_name1}{dataset_suffix}.csv')
output_file2 = os.path.join(base_path, f'{dataset_name2}{dataset_suffix}.csv')
output_file3 = os.path.join(base_path, f'{dataset_name3}{dataset_suffix}.csv')
output_file4 = os.path.join(base_path, f'{dataset_name4}{dataset_suffix}.csv')
output_file5 = os.path.join(base_path, f'{dataset_name5}{dataset_suffix}.csv')
# Load the newly generated CSV file
df1 = pd.read_csv(output_file1)
df2 = pd.read_csv(output_file2)
df3 = pd.read_csv(output_file3)
df4 = pd.read_csv(output_file4)
df5 = pd.read_csv(output_file5)
# add df1 to df
df = pd.concat([df1,df2,df3,df4,df5], ignore_index=True)
# print the number of rows
print(f"Number of rows: {len(df)}")
df = df.dropna()
print(df.shape)

Number of rows: 14749502


In [ ]:
df = df[df["Crop_type"] == crop_selected]
print(f"Number of rows: {len(df)}")

In [ ]:
# Visualize the features in the dataset
features = df.columns
# Print the number of predictors
print(f'The number of features is: {len(features)}')
# List the potential predictors
print(features)

In [ ]:
# print unique values in 'Irrigation' and their counts
print(df['Irrigation'].value_counts())

In [ ]:
# add a new column 'Irrigation_code' to the dataframe, 1 for Irrigated and 0 for Rainfed
df['Irrigation_code'] = df['Irrigation'].apply(lambda x: 1 if x == 'Irrigated' else 0) 
# print the number of rows with Irrigation_code = 1
print(f"Number of rows with Irrigation_code = 1: {len(df[df['Irrigation_code'] == 1])}")
# print the number of rows with Irrigation_code = 0
print(f"Number of rows with Irrigation_code = 0: {len(df[df['Irrigation_code'] == 0])}")

In [ ]:
# Visualize the features in the dataset
features = df.columns
# Print the number of predictors
print(f'The number of features is: {len(features)}')
# List the potential predictors
print(features)

In [ ]:
# Investigate value counts for each class in Crop_type
print(df['Crop_type'].value_counts())

In [ ]:
# separate df irrigated and rainfed
df_irrigated = df[df["Irrigation"] == "Irrigated"]
df_rainfed = df[df["Irrigation"] == "Rainfed"]
# print the number of rows in each dataframe
print(f"Number of rows in df_irrigated: {len(df_irrigated)}")
print(f"Number of rows in df_rainfed: {len(df_rainfed)}")

In [ ]:
# --- Common preprocessing function ---
def preprocess(df):
    df = df.copy()
    df["week"] = pd.to_numeric(df["week"], errors="coerce")
    df["veg_mean_ndvi"] = pd.to_numeric(df["veg_mean_ndvi"], errors="coerce")
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    df = df.dropna(subset=["Year", "week"])
    df = df[df["week"] > 0]

    def iso_to_date(year, week):
        try:
            return datetime.fromisocalendar(int(year), int(week), 1)
        except:
            return pd.NaT

    df["date"] = df.apply(lambda row: iso_to_date(row["Year"], row["week"]), axis=1)
    return df.dropna(subset=["date"])

# --- Preprocess both datasets ---
df_irrigated_clean = preprocess(df_irrigated)
df_rainfed_clean = preprocess(df_rainfed)

# --- Group and average by date ---
avg_irrigated = df_irrigated_clean.groupby("date", as_index=False)["veg_mean_ndvi"].mean()
avg_rainfed = df_rainfed_clean.groupby("date", as_index=False)["veg_mean_ndvi"].mean()


# --- Plotting ---
plt.figure(figsize=(14, 7))

# Rainfed line
plt.plot(
    avg_rainfed["date"].to_numpy(),
    avg_rainfed["veg_mean_ndvi"].to_numpy(),
    marker="s",
    linestyle="--",
    color="orange",
    label=f"Rainfed"
)

# Irrigated line
plt.plot(
    avg_irrigated["date"].to_numpy(),
    avg_irrigated["veg_mean_ndvi"].to_numpy(),
    marker="o",
    linestyle="-",
    color="blue",
    label=f"Irrigated"
)

plt.xlabel("Date")
plt.ylabel("Average veg_mean_ndvi")
plt.title(f"NDVI Time Series for {crop_selected} (Irrigated vs. Rainfed)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.ylim(0.1, 0.8)
plt.show()

In [ ]:
# Make a safe copy to avoid SettingWithCopyWarning
df_rainfed = df_rainfed.copy()

# Ensure 'week', 'veg_mean_ndvi', and 'Year' are numeric
df_rainfed["week"] = pd.to_numeric(df_rainfed["week"], errors="coerce")
df_rainfed["veg_mean_ndvi"] = pd.to_numeric(df_rainfed["veg_mean_ndvi"], errors="coerce")
df_rainfed["Year"] = pd.to_numeric(df_rainfed["Year"], errors="coerce")

# Compute average NDVI for crop_selected only, per Year and Week
avg_ndvi_crop_selected = df_rainfed.groupby(["Year", "week"], as_index=False)["veg_mean_ndvi"].mean()

# Create the plot
plt.figure(figsize=(14, 7))

# Line styles and color map
linestyles = itertools.cycle(["-", "--", "-.", ":"])
colors = plt.cm.tab10.colors

# Plot crop_selected per year
for i, year in enumerate(avg_ndvi_crop_selected["Year"].unique()):
    subset = avg_ndvi_crop_selected[avg_ndvi_crop_selected["Year"] == year].sort_values("week")
    weeks = subset["week"].values
    ndvi_values = subset["veg_mean_ndvi"].values

    plt.plot(
        weeks,
        ndvi_values,
        marker="o",
        linestyle=next(linestyles),
        color=colors[i % len(colors)],
        label=f"{int(year)}"

    )

# Customize the plot
plt.xlabel("Week")
plt.ylabel("Average veg_mean_ndvi")
plt.title("Weekly Average veg_mean_ndvi for "+crop_selected+" Rainfed (by Year)")
plt.legend(title="Year", loc='upper right', frameon=True)
plt.grid(True)
plt.tight_layout()
plt.ylim(.1, .8)


# Show the plot
plt.show()


In [ ]:
# Make a safe copy to avoid SettingWithCopyWarning
df_irrigated = df_irrigated.copy()

# Ensure 'week', 'veg_mean_ndvi', and 'Year' are numeric
df_irrigated["week"] = pd.to_numeric(df_irrigated["week"], errors="coerce")
df_irrigated["veg_mean_ndvi"] = pd.to_numeric(df_irrigated["veg_mean_ndvi"], errors="coerce")
df_irrigated["Year"] = pd.to_numeric(df_irrigated["Year"], errors="coerce")

# Compute average NDVI for crop_selected only, per Year and Week
avg_ndvi_crop_selected = df_irrigated.groupby(["Year", "week"], as_index=False)["veg_mean_ndvi"].mean()

# Create the plot
plt.figure(figsize=(14, 7))

# Line styles and color map
linestyles = itertools.cycle(["-", "--", "-.", ":"])
colors = plt.cm.tab10.colors

# Plot crop_selected per year
for i, year in enumerate(avg_ndvi_crop_selected["Year"].unique()):
    subset = avg_ndvi_crop_selected[avg_ndvi_crop_selected["Year"] == year].sort_values("week")
    weeks = subset["week"].values
    ndvi_values = subset["veg_mean_ndvi"].values

    plt.plot(
        weeks,
        ndvi_values,
        marker="o",
        linestyle=next(linestyles),
        color=colors[i % len(colors)],
        label=f"{int(year)}"

    )

# Customize the plot
plt.xlabel("Week")
plt.ylabel("Average veg_mean_ndvi")
plt.title("Weekly Average veg_mean_ndvi for "+crop_selected+" Irrigated (by Year)")
plt.legend(title="Year", loc='upper right', frameon=True)
plt.grid(True)
plt.tight_layout()
plt.ylim(.1, .8)


# Show the plot
plt.show()

In [ ]:
# plot pie chart of unique parcels belonging to each irrigation type
def plot_pie_chart(df, title):
    # Count unique parcels for each irrigation type
    counts = df['Irrigation'].value_counts()

    # Define custom colors: blue for Irrigated, orange for Rainfed
    color_map = {'Irrigated': 'lightblue', 'Rainfed': 'orange'}
    # Match the color order to the index order
    colors = [color_map.get(label, 'gray') for label in counts.index]

    # Create a pie chart
    plt.figure(figsize=(5, 5))
    plt.pie(counts, labels=counts.index, colors=colors,
            autopct='%1.1f%%', startangle=140)
    plt.title(title)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

# Example usage
plot_pie_chart(df, f"{crop_selected}")

